In [16]:
from hashlib import sha1
import hmac
import binascii

def generate_full_url(route_name):
    devId = 3002823
    key = '23301ed1-37b4-4eea-a384-43d667411ee9'
    base_url = 'https://timetableapi.ptv.vic.gov.au'
    request_path = '/v3/routes?route_types=2&route_name={}'.format(route_name)
    full_request = request_path + ('&' if '?' in request_path else '?') + 'devid={}'.format(devId)
    hashed = hmac.new(key.encode(), full_request.encode(), sha1)
    signature = hashed.hexdigest()
    complete_url = '{}{}&signature={}'.format(base_url, full_request, signature)
    return complete_url

# 示例使用，动态输入路线名称
route_name = "Clockwise to Clockwise"
print(generate_full_url(route_name))


https://timetableapi.ptv.vic.gov.au/v3/routes?route_types=2&route_name=Clockwise to Clockwise&devid=3002823&signature=ba145fd695e29c09d30a1d717386d9f9eae41792


In [2]:
from hashlib import sha1
import hmac
import binascii
from urllib.parse import quote  # Import the URL encoding function

def getUrl(route_type, route_name):
    devId = 3002823  # Your developer ID
    key = '23301ed1-37b4-4eea-a384-43d667411ee9'  # Your developer key

    # Encode the route name to handle URL special characters like spaces
    encoded_route_name = quote(route_name)

    # Construct the request query
    request = f'v3/routes?route_types=1&devid=3002823'

    # Append devId to the request query
    raw = request + '&devid={0}'.format(devId)

    # Calculate HMAC-SHA1 signature
    hashed = hmac.new(key.encode(), raw.encode(), sha1)
    signature = hashed.hexdigest()

    # Return the complete URL with signature
    return 'https://timetableapi.ptv.vic.gov.au' + raw + '&signature={1}'.format(devId, signature)

# Example usage
print(getUrl(1, 'City Circle (Free Tourist Tram)'))


https://timetableapi.ptv.vic.gov.au/v3/routes?route_types=1&devid=3002823&devid=3002823&signature=fdb41523e6a8314fe0ddcf6f298b3693a3e3f84d


In [27]:
import gtfs_kit as gk
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString

# 载入GTFS数据
feed = gk.read_feed('gtfs.zip', dist_units='m')

import geopandas as gpd
from shapely.geometry import Point

# 假设A点坐标
point_a = Point(144.955482,-37.820819)
point_b = Point(144.964422,-37.809364)

# 创建地理数据框
stations = gpd.GeoDataFrame(feed.stops, geometry=gpd.points_from_xy(feed.stops.stop_lon, feed.stops.stop_lat))
def calculate_nearest_station(point):
    stations['distance'] = stations.geometry.distance(point)
    nearest_station = stations.loc[stations['distance'].idxmin()]
    return nearest_station
station_a=calculate_nearest_station(point_a)
station_b=calculate_nearest_station(point_b)



In [56]:
a_station_id = station_a.stop_id
b_station_id = station_b.stop_id

a_trips = set(feed.stop_times[feed.stop_times['stop_id'] == a_station_id]['trip_id'])
b_trips = set(feed.stop_times[feed.stop_times['stop_id'] == b_station_id]['trip_id'])

# 直达路线的行程 ID
direct_trip_ids = a_trips.intersection(b_trips)

# 初始化最短行程变量
min_duration = float('inf')
shortest_trip = None

# 遍历直达行程
for trip_id in direct_trip_ids:
    trip_stops = feed.stop_times[feed.stop_times['trip_id'] == trip_id].sort_values('stop_sequence')
    start_index = trip_stops[trip_stops['stop_id'] == a_station_id].index.min()
    end_index = trip_stops[trip_stops['stop_id'] == b_station_id].index.min()
    print('start_index:',start_index)
    print('end_index:',end_index)
    
    # 确保方向正确，取决于 start_index 和 end_index 的关系
    if start_index < end_index:  # 正向行程
        relevant_stops = trip_stops.loc[start_index:end_index]
    elif start_index > end_index:  # 反向行程
        relevant_stops = trip_stops.loc[end_index:start_index]
    else:
        continue  # 如果起点和终点相同

    # 计算行程总持续时间
    departure_time = pd.to_timedelta(relevant_stops.iloc[0]['departure_time'])
    arrival_time = pd.to_timedelta(relevant_stops.iloc[-1]['arrival_time'])
    duration = (arrival_time - departure_time).total_seconds()

    # 更新最短行程
    if duration < min_duration:
        min_duration = duration
        shortest_trip = trip_id, relevant_stops

# 如果找到最短行程
if shortest_trip:
    trip_id, stops = shortest_trip
    print(f"最短直达路线行程ID: {trip_id}, 持续时间: {min_duration/60} 分钟")
    for _, stop in stops.iterrows():
        stop_name = feed.stops[feed.stops['stop_id'] == stop['stop_id']].iloc[0]['stop_name']
        print(f"{stop['stop_id']} - {stop_name}")
else:
    print("没有找到直达路线，需要考虑换乘")

start_index: 1055889
end_index: 1055876
start_index: 827267
end_index: 827254
start_index: 774766
end_index: 774753
start_index: 1067909
end_index: 1067896
start_index: 571928
end_index: 571915
start_index: 498492
end_index: 498479
start_index: 122736
end_index: 122723
start_index: 687576
end_index: 687563
start_index: 503498
end_index: 503485
start_index: 425866
end_index: 425853
start_index: 774796
end_index: 774783
start_index: 381881
end_index: 381868
start_index: 670426
end_index: 670413
start_index: 743368
end_index: 743355
start_index: 951237
end_index: 951224
start_index: 501544
end_index: 501531
start_index: 573974
end_index: 573961
start_index: 776965
end_index: 776952
start_index: 427859
end_index: 427846
start_index: 64406
end_index: 64393
start_index: 1588
end_index: 1575
start_index: 1039850
end_index: 1039837
start_index: 287210
end_index: 287197
start_index: 1117639
end_index: 1117626
start_index: 1021112
end_index: 1021099
start_index: 651394
end_index: 651381


KeyboardInterrupt: 

In [72]:
def get_route_id_from_trip(feed, trip_id):
    # 从 trips 表中查询指定 trip_id 的 route_id
    route_id = feed.trips[feed.trips['trip_id'] == trip_id]['route_id'].iloc[0]
    return route_id

# 示例：假设你已知的 trip_id
trip_id = "2.T0.3-35-mjp-4.1.H"

# 调用函数，获取 route_id
route_id = get_route_id_from_trip(feed, trip_id)
print(f"对应的 route_id 为: {route_id}")


对应的 route_id 为: 3-35-mjp-4


In [74]:
import requests
import gtfs_realtime_pb2

def parse_vehicle_data(trip_id):
    url = "https://data-exchange-api.vicroads.vic.gov.au/opendata/gtfsr/v1/tram/vehicleposition"
    headers = {"Ocp-Apim-Subscription-Key": "af02fa9cf4764385ad592b462db4ec8e"}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        feed = gtfs_realtime_pb2.FeedMessage()
        feed.ParseFromString(response.content)

        for entity in feed.entity:
            if entity.HasField('vehicle') and entity.vehicle.trip.trip_id == trip_id:
                vehicle = entity.vehicle
                return {
                    "Vehicle ID": vehicle.vehicle.id,
                    "Vehicle Label": vehicle.vehicle.label,
                    "Latitude": vehicle.position.latitude,
                    "Longitude": vehicle.position.longitude
                }
    return None

In [78]:
vehicle_info = parse_vehicle_data('46.T0.3-86-mjp-4.3.H')
print(vehicle_info)

None


In [1]:
import gtfs_realtime_pb2
import requests

def parse_vehicle_data_for_trip_id(data, trip_id_to_find):
    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(data)

    inaccessible_models = ['Z1', 'Z3', 'A1', 'A2']
    for entity in feed.entity:
        if entity.HasField('vehicle'):
            vehicle = entity.vehicle
            trip = vehicle.trip
            if trip.trip_id == trip_id_to_find:  # Check if the current trip ID matches the desired trip ID
                position = vehicle.position
                vehicle_details = vehicle.vehicle
                # Check if the vehicle label contains any of the specified inaccessible models
                accessible = 0 if any(model in vehicle_details.label for model in inaccessible_models) else 1
                vehicle_info = {
                    "Vehicle ID": vehicle_details.id,
                    "Vehicle Label": vehicle_details.label,
                    "Accessible": accessible  # Add accessibility info
                }
                return vehicle_info  # Return the found vehicle info immediately
    return None  # Return None if no vehicle matches the specified trip ID

# Get data
url = "https://data-exchange-api.vicroads.vic.gov.au/opendata/gtfsr/v1/tram/vehicleposition"
headers = {
    "Ocp-Apim-Subscription-Key": "af02fa9cf4764385ad592b462db4ec8e"
}
response = requests.get(url, headers=headers)

specific_trip_id = "264.291.T0.3-78-mjp-1.2.R"  # The trip ID you are interested in
vehicle_info = parse_vehicle_data_for_trip_id(response.content, specific_trip_id)
if vehicle_info:
    print(vehicle_info['Accessible'])
else:
    vehicle_info=1
    print(vehicle_info)


1


In [95]:
import pandas as pd
import datetime
from datetime import timedelta

def parse_time(t):
    if pd.isna(t):
        return None
    hours, minutes, seconds = map(int, t.split(':'))
    return timedelta(hours=hours, minutes=minutes, seconds=seconds)

def find_next_departures(feed, a_station_id, b_station_id, hours_offset=10):
    now = datetime.datetime.now() - timedelta(hours=hours_offset)
    now_time = now.time()
    now_delta = timedelta(hours=now_time.hour, minutes=now_time.minute, seconds=now_time.second)
    print(f"Adjusted Current Time Delta: {now_delta}")

    a_trips = set(feed.stop_times[feed.stop_times['stop_id'] == a_station_id]['trip_id'])
    b_trips = set(feed.stop_times[feed.stop_times['stop_id'] == b_station_id]['trip_id'])
    direct_trip_ids = a_trips.intersection(b_trips)

    min_duration = float('inf')
    shortest_trip = None

    for trip_id in direct_trip_ids:
        trip_stops = feed.stop_times[feed.stop_times['trip_id'] == trip_id].sort_values('stop_sequence')
        start_index = trip_stops[trip_stops['stop_id'] == a_station_id].index.min()
        end_index = trip_stops[trip_stops['stop_id'] == b_station_id].index.min()
        accessible = parse_vehicle_data_for_trip_id(response.content,trip_id)
        if accessible:
            accessible = accessible['Accessible']
        else:
            accessible = 1

        if pd.isna(start_index) or pd.isna(end_index):
            print("Skipping trip due to NA index.")
            continue

        if start_index < end_index:
            relevant_stops = trip_stops.loc[start_index:end_index]
        elif start_index > end_index:
            relevant_stops = trip_stops.loc[end_index:start_index]
        else:
            continue

        if not relevant_stops.empty:
            departure_time_str = relevant_stops.iloc[0]['departure_time']
            departure_time = parse_time(departure_time_str)
            arrival_time_str = relevant_stops.iloc[-1]['arrival_time']
            arrival_time = parse_time(arrival_time_str)

            if departure_time and arrival_time and departure_time > now_delta:
                duration = (arrival_time - departure_time).total_seconds()
                if duration < min_duration:
                    min_duration = duration
                    shortest_trip = trip_id, departure_time_str, arrival_time_str, duration / 60, accessible  # Storing more info here

    return shortest_trip

# Example usage

nearest_trip = find_next_departures(feed, a_station_id, b_station_id)
if nearest_trip:
    trip_id, departure_time_str, arrival_time_str, duration_minutes,accessible = nearest_trip
    print(f"Next departure trip ID: {trip_id}, Departure at: {departure_time_str}, Arrives at: {arrival_time_str}, Duration: {duration_minutes} minutes, accessible: {accessible}")
    for _, stop in stops.iterrows():
        stop_name = feed.stops[feed.stops['stop_id'] == stop['stop_id']].iloc[0]['stop_name']
        print(f"{stop['stop_id']} - {stop_name}")
else:
    print("No upcoming trips found")


Adjusted Current Time Delta: 15:45:32
Next departure trip ID: 5.T0.3-35-mjp-4.1.H, Departure at: 17:04:00, Arrives at: 17:31:00, Duration: 27.0 minutes, accessible: 1
18046 - 6-Swanston St/La Trobe St (Melbourne City)
18047 - 7-Russell St/La Trobe St (Melbourne City)
18048 - 8-Exhibition St/La Trobe St (Melbourne City)
18049 - 9-Victoria St/La Trobe St (Melbourne City)
18050 - 10-Nicholson St/Victoria Pde (Carlton)
17880 - 10-Albert St/Nicholson St (Fitzroy)
20979 - 0-Bourke St/Spring St (Melbourne City)
18105 - 8-Spring St/Flinders St (Melbourne City)
18174 - 6-Russell St/Flinders St (Melbourne City)
18175 - 5-Swanston St/Flinders St (Melbourne City)
18176 - 4-Elizabeth St/Flinders St (Melbourne City)
18178 - 3-Market St/Flinders St (Melbourne City)
21233 - 2-Melbourne Aquarium/Flinders St (Melbourne City)
21232 - 1-Spencer St/Flinders St (Melbourne City)


In [97]:
stops

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
977806,5.T0.3-35-mjp-4.1.H,17:04:00,17:04:00,18046,10,NaN,0,0,2547.02
977807,5.T0.3-35-mjp-4.1.H,17:05:00,17:05:00,18047,11,NaN,0,0,2696.79
977808,5.T0.3-35-mjp-4.1.H,17:06:00,17:06:00,18048,12,NaN,0,0,2926.67
977809,5.T0.3-35-mjp-4.1.H,17:07:00,17:07:00,18049,13,NaN,0,0,3088.49
977810,5.T0.3-35-mjp-4.1.H,17:08:00,17:08:00,18050,14,NaN,0,0,3322.60
977811,5.T0.3-35-mjp-4.1.H,17:09:00,17:09:00,17880,15,NaN,0,0,3551.14
977812,5.T0.3-35-mjp-4.1.H,17:10:00,17:10:00,20979,16,NaN,0,0,3783.18
977813,5.T0.3-35-mjp-4.1.H,17:20:00,17:20:00,18105,17,NaN,0,0,4274.58
977814,5.T0.3-35-mjp-4.1.H,17:22:00,17:22:00,18174,18,NaN,0,0,4648.26
977815,5.T0.3-35-mjp-4.1.H,17:24:00,17:24:00,18175,19,NaN,0,0,4993.06


In [24]:
# # 分析经过每个站点的路线
# routes_at_stops = feed.stop_times.merge(feed.trips[['trip_id', 'route_id']], on='trip_id')
# routes_at_stops = routes_at_stops.groupby('stop_id')['route_id'].apply(set)

# # 查找直达路线和换乘点
# routes_from_a = routes_at_stops[station_a['stop_id']]
# routes_to_b = routes_at_stops[station_b['stop_id']]

# # 寻找直达路线
# direct_routes = routes_from_a.intersection(routes_to_b)

# # 如果没有直达路线，寻找可能的换乘点
# if not direct_routes:
#     transfer_stations = []
#     for stop_id, routes in routes_at_stops.items():
#         if routes_from_a.intersection(routes) and routes_to_b.intersection(routes):
#             transfer_stations.append(stop_id)


NameError: name 'station_a' is not defined

In [ ]:
# import gtfs_kit as gk
# import pandas as pd
# import geopandas as gpd
# from shapely.geometry import Point

# # 载入GTFS数据
# feed = gk.read_feed('gtfs.zip', dist_units='m')

# # 创建地理数据框
# stations = gpd.GeoDataFrame(feed.stops, geometry=gpd.points_from_xy(feed.stops.stop_lon, feed.stops.stop_lat))

# def calculate_nearest_station(point):
#     stations['distance'] = stations.geometry.distance(point)
#     return stations.loc[stations['distance'].idxmin()]

# # 假设 A 点和 B 点坐标
# point_a = Point(144.955482,-37.820819)
# point_b = Point(144.964422,-37.809364)

# # 计算 A 点和 B 点最近的站点
# station_a = calculate_nearest_station(point_a)
# station_b = calculate_nearest_station(point_b)

# def find_shortest_trip(station_a, station_b):
#     a_station_id = station_a.stop_id
#     b_station_id = station_b.stop_id

#     a_trips = set(feed.stop_times[feed.stop_times['stop_id'] == a_station_id]['trip_id'])
#     b_trips = set(feed.stop_times[feed.stop_times['stop_id'] == b_station_id]['trip_id'])

#     # 直达路线的行程 ID
#     direct_trip_ids = a_trips.intersection(b_trips)

#     # 初始化最短行程变量
#     min_duration = float('inf')
#     shortest_trip = None

#     # 遍历直达行程
#     for trip_id in direct_trip_ids:
#         trip_stops = feed.stop_times[feed.stop_times['trip_id'] == trip_id].sort_values('stop_sequence')
#         start_index = trip_stops[trip_stops['stop_id'] == a_station_id].index.min()
#         end_index = trip_stops[trip_stops['stop_id'] == b_station_id].index.min()

#         if start_index < end_index:  # 正向行程
#             relevant_stops = trip_stops.loc[start_index:end_index]
#         else:
#             continue  # 如果行程是反向的或起点终点相同，跳过

#         # 计算行程总持续时间
#         departure_time = pd.to_timedelta(relevant_stops.iloc[0]['departure_time'])
#         arrival_time = pd.to_timedelta(relevant_stops.iloc[-1]['arrival_time'])
#         duration = (arrival_time - departure_time).total_seconds()

#         # 更新最短行程
#         if duration < min_duration:
#             min_duration = duration
#             shortest_trip = trip_id, relevant_stops

#     return shortest_trip

# # 执行搜索最短行程
# shortest_trip = find_shortest_trip(station_a, station_b)

# if shortest_trip:
#     trip_id, stops = shortest_trip
#     print(f"最短直达路线行程ID: {trip_id}, 持续时间: {min_duration/60} 分钟")
#     for _, stop in stops.iterrows():
#         stop_name = feed.stops[feed.stops['stop_id'] == stop['stop_id']].iloc[0]['stop_name']
#         print(f"{stop['stop_id']} - {stop_name}")
# else:
#     print("没有找到直达路线，需要考虑换乘")
